In [ ]:
from google.colab import drive, output
drive.mount('/content/drive')
import sys
!pip install torchaudio
!pip install pythae
output.clear()
#%pdb on

%load_ext autoreload
%autoreload 1
%cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
#sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/data')
#sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/models')
sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torchaudio.transforms as audio_transform
import pythae
from pythae.pipelines import TrainingPipeline
from pythae.models import VQVAE, VQVAEConfig
from pythae.models import VAE, VAEConfig
from pythae.models import VAE_LinNF, VAE_LinNF_Config
from pythae.models import AE, AEConfig
from pythae.models import MIWAE, MIWAEConfig
from pythae.trainers import BaseTrainerConfig

from Pythae_DataLoader import SoundscapeData

/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project


In [ ]:
my_training_config = BaseTrainerConfig(
    output_dir='my_model_VAE',
    num_epochs=3,
    learning_rate=1e-5,
    batch_size=12,
    steps_saving=None
)

In [ ]:
my_vae_config = model_config = VAEConfig(
    input_dim=(1, 515, 515),
    latent_dim=256,
)



In [ ]:
my_vae_model = VAE(
    model_config=my_vae_config
)

In [ ]:
pipeline = TrainingPipeline(
    training_config=my_training_config,
    model=my_vae_model
)

In [ ]:
root_path = '/content/drive/Shareddrives/ConservacionBiologicaIA/Datos/Jaguas_2018'


dataset = SoundscapeData(root_path, audio_length=12, ext="wav", win_length=1028)
dataset_train, dataset_test = random_split(dataset,
                                           [round(len(dataset)*0.7), len(dataset) - round(len(dataset)*0.7)], 
                                           generator=torch.Generator().manual_seed(1024))

In [ ]:
iterator = iter(dataset_train)
dictado = next(iterator)
type(dictado.data)
dictado.data.shape

torch.Size([1, 515, 515])

In [ ]:
pipeline(
    train_data=dataset_train, # must be torch.Tensor or np.array
    eval_data=dataset_test# must be torch.Tensor or np.array
)


Checking train dataset...
INFO:pythae.pipelines.training:Checking train dataset...
Checking eval dataset...
INFO:pythae.pipelines.training:Checking eval dataset...
Using Base Trainer

INFO:pythae.pipelines.training:Using Base Trainer

Model passed sanity check !

INFO:pythae.trainers.base_trainer.base_trainer:Model passed sanity check !

Created my_model_VAE/VAE_training_2022-10-17_00-29-03. 
Training config, checkpoints and final model will be saved here.

INFO:pythae.trainers.base_trainer.base_trainer:Created my_model_VAE/VAE_training_2022-10-17_00-29-03. 
Training config, checkpoints and final model will be saved here.

Successfully launched training !

INFO:pythae.trainers.base_trainer.base_trainer:Successfully launched training !



Training of epoch 1/3:   0%|          | 0/1171 [00:00<?, ?batch/s]

Eval of epoch 1/3:   0%|          | 0/502 [00:00<?, ?batch/s]

--------------------------------------------------------------------------
INFO:pythae.trainers.base_trainer.base_trainer:--------------------------------------------------------------------------
Train loss: 75076.4158
INFO:pythae.trainers.base_trainer.base_trainer:Train loss: 75076.4158
Eval loss: 61675.0162
INFO:pythae.trainers.base_trainer.base_trainer:Eval loss: 61675.0162
--------------------------------------------------------------------------
INFO:pythae.trainers.base_trainer.base_trainer:--------------------------------------------------------------------------


Training of epoch 2/3:   0%|          | 0/1171 [00:00<?, ?batch/s]

## Using the trained model and samplers

In [ ]:
import os
from pythae.models import AutoModel

In [ ]:
last_training = sorted(os.listdir("my_model"))[-1]
trained_model = AutoModel.load_from_folder(os.path.join("my_model", last_training, "final_model"))

## Using wandb logging

In [ ]:
from pythae.trainers.training_callbacks import WandbCallback

callbacks = [] # the TrainingPipeline expects a list of callbacks

wandb_cb = WandbCallback() # Build the callback 

# SetUp the callback 
wandb_cb.setup(
    training_config=training_config, # training config
    model_config=model_config, # model config
    project_name="your_wandb_project", # specify your wandb project
    entity_name="your_wandb_entity", # specify your wandb entity
)

callbacks.append(wandb_cb) # Add it to the callbacks list